In [1]:
from datetime import datetime
import pandas as pd
import warnings
import math
import sys
sys.path.append('../src')
from functions import stock_selection_weight_allocation, adjust_portfolio, calculate_shares_to_buy_with_prices
warnings.filterwarnings("ignore")

In [2]:
buy_date_str = '2024-02-26'
buy_date = pd.to_datetime(buy_date_str)

stock_selection_strategy = 13
weight_allocation_strategy = 6

# backtest version 5
last_x_years = 0.5
last_x_years_opt = 0.5

In [3]:
all_stocks_df = pd.read_csv('../data/all_stock_data.csv', index_col=0)
all_stocks_df.index = pd.to_datetime(all_stocks_df.index)
all_stocks_df

,360ONE.NS,3MINDIA.NS,ABB.NS,ACC.NS,AGI.NS,AIAENG.NS,APLAPOLLO.NS,AUBANK.NS,AARTIDRUGS.NS,AARTIIND.NS,...,ZEEL.NS,ZENTEC.NS,ZENSARTECH.NS,ZOMATO.NS,ZYDUSLIFE.NS,ZYDUSWELL.NS,ECLERX.NS,EMUDHRA.NS,ACLGATI.NS,WELSPUNLIV.NS
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,264.346344,1779.729614,650.601746,722.450684,9.885830,333.858276,12.891179,282.469299,10.365798,8.898396,...,105.713058,53.752098,24.538420,126.000000,77.028969,243.609695,120.262016,256.129639,57.730717,7.873486
2010-01-05,264.346344,1779.393066,648.944153,713.079895,10.383352,343.307556,12.891179,282.469299,10.268604,8.898396,...,106.864403,53.752098,24.083796,126.000000,79.013733,243.020615,126.622154,256.129639,57.596756,7.687500
2010-01-06,264.346344,1769.392090,649.156616,717.706116,10.318739,333.007385,12.891179,282.469299,10.419255,8.880511,...,109.229851,53.752098,24.190123,126.000000,79.698532,241.162704,126.735474,256.129639,57.105633,7.819755
2010-01-07,264.346344,1783.816528,664.966919,722.252991,10.835644,322.438599,12.891179,282.469299,10.258885,8.817906,...,109.292633,53.752098,24.237785,126.000000,79.750763,238.217316,126.735474,256.129639,57.418175,7.699896
2010-01-08,264.346344,1779.585327,676.612183,721.304138,11.171633,331.081726,12.891179,282.469299,10.560185,8.853681,...,107.659851,53.752098,23.944483,126.000000,80.215050,237.537567,133.463913,256.129639,57.775345,7.766028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-19,702.000000,30918.400391,4488.850098,2670.500000,819.150024,3922.649902,1431.500000,599.349976,541.150024,689.049988,...,178.500000,805.650024,532.849976,157.500000,907.549988,1597.550049,2312.300049,532.000000,114.500000,153.100006
2024-02-20,705.500000,31384.599609,4524.500000,2686.699951,831.849976,3876.250000,1439.550049,599.099976,543.650024,703.900024,...,192.649994,814.599976,527.849976,161.050003,895.750000,1595.300049,2404.149902,558.099976,121.449997,157.100006
2024-02-21,713.700012,31009.250000,4984.700195,2666.750000,801.650024,3797.699951,1433.949951,588.150024,533.450012,687.400024,...,164.600006,806.450012,528.200012,158.649994,893.799988,1595.449951,2449.050049,568.900024,119.099998,154.000000


In [4]:
govt_bond_df = pd.read_csv('../data/India 10-Year Bond Yield Historical Data.csv')
govt_bond_df = govt_bond_df[['Date','Price']]
govt_bond_df.index = govt_bond_df['Date']
govt_bond_df = govt_bond_df.drop('Date', axis=1)
govt_bond_df.index = pd.to_datetime(govt_bond_df.index)
govt_bond_df

,Price
Date,
2024-01-01,7.196
2023-12-29,7.176
2023-12-28,7.207
2023-12-27,7.205
2023-12-26,7.183
...,...
2020-01-07,6.550
2020-01-06,6.565
2020-01-03,6.510


In [5]:
filters = 4

if stock_selection_strategy in [1,2,5,6,9,10,13,14]:
    holding_period = '1q'
elif stock_selection_strategy in [3,4,7,8,11,12,15,16]:
    holding_period = '1m'

if stock_selection_strategy in [1,3,5,7,9,11,13,15]:
    returns_type = 'SR'
elif stock_selection_strategy in [2,4,6,8,10,12,14,16]:
    returns_type = 'LR'

if stock_selection_strategy in [1,2,3,4]:
    max_non_positive_returns_count = 15
elif stock_selection_strategy in [5,6,7,8]:
    max_non_positive_returns_count = 10
elif stock_selection_strategy in [9,10,11,12]:
    max_non_positive_returns_count = None
    filters = 3
elif stock_selection_strategy in [13,14,15,16]:
    max_non_positive_returns_count = None
    filters = 2

In [6]:
portfolio, sell_date, best_method = stock_selection_weight_allocation(buy_date_str, holding_period, returns_type, max_non_positive_returns_count, weight_allocation_strategy, all_stocks_df, govt_bond_df, filters, last_x_years, last_x_years_opt)        
portfolio = adjust_portfolio(portfolio)
portfolio

{'ANANDRATHI.NS': 0.125,
 'PRESTIGE.NS': 0.125,
 'GET&D.NS': 0.125,
 'SWANENERGY.NS': 0.125,
 'TATAINVEST.NS': 0.125,
 'BSE.NS': 0.125,
 'OIL.NS': 0.125,
 'SOBHA.NS': 0.125}

In [7]:
total_investment = 20000

In [8]:
adjusted_buy_date = all_stocks_df[all_stocks_df.index <= buy_date].index[-1]

In [9]:
final_shares, price_dict, actual_investment = calculate_shares_to_buy_with_prices(portfolio, all_stocks_df, adjusted_buy_date, total_investment, strictly_lower=False)

In [10]:
final_shares, price_dict, actual_investment

({'PRESTIGE.NS': 2,
  'GET&D.NS': 4,
  'SWANENERGY.NS': 4,
  'BSE.NS': 1,
  'OIL.NS': 5,
  'SOBHA.NS': 2},
 {'PRESTIGE.NS': 1230.4000244140625,
  'GET&D.NS': 827.2000122070312,
  'SWANENERGY.NS': 736.5,
  'BSE.NS': 2276.800048828125,
  'OIL.NS': 591.5999755859375,
  'SOBHA.NS': 1460.199951171875},
 16870.799926757812)